In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns 
import re
from tqdm.notebook import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

cafeterias = pd.read_csv('database/cafeteria.csv',sep=';')
restaurantes = pd.read_csv('database/restaurante.csv',sep=';')
padarias = pd.read_csv('database/padaria.csv',sep=';')
delivery = pd.read_csv('database/delivery de comida.csv',sep=';')
driveThru = pd.read_csv('database/drive thru.csv',sep=';')

bruto = pd.concat([cafeterias,restaurantes,padarias,delivery,driveThru],ignore_index=True)

bruto.dropna(subset=['LOCAL'], inplace=True)

bruto['NUM_COMENTARIO'] = bruto['COMENTARIO'].str.replace('comentários', '', regex=False).str.replace(' ', '')
bruto['NUM_COMENTARIO'] = bruto['NUM_COMENTARIO'].fillna(0)
bruto['PONTUACAO'] = bruto['PONTUACAO'].fillna(0)

def extrair_hora(texto):
    try:
        texto = texto.replace(',','.').strip().lower()
        tempos = []
        partes = texto.split('a') if 'a' in texto else [texto]

        for parte in partes:
            parte = parte.strip()
            if 'min' in parte:
                minutos = float(re.search(r'(\d+\.?\d*)\s*min', parte).group(1))
            elif 'h'in partes:
                minutos = float(re.search(r'(\d+\.?\d*)\s*h', parte).group(1))
            else:
                minutos = None
            if minutos is not None:
                tempos.append(minutos)
        if len(tempos) == 1:
            return pd.Series([tempos[0], tempos[0]])
        else:
            return pd.Series([min(tempos), max(tempos)])
    except:
        return pd.Series([None, None])

        
bruto[['tempo_min_minutos', 'tempo_max_minutos']] = bruto['TEMPO_ESPERA'].apply(extrair_hora).fillna(0)

bruto

,NOME,PONTUACAO,COMENTARIO,LOCAL,TIPO,TEMPO_ESPERA,BUSCA_ID,NUM_COMENTARIO,tempo_min_minutos,tempo_max_minutos
0,Shopping RioMar Fortaleza,4.7,52.356 comentários,"R. Des. Lauro Nogueira, 1500 - Papicu, Fortale...","Shopping center em Fortaleza, Ceará",45 min a 3 h,cafeteria,52.356,45.0,45.0
1,Greta Café,4.7,1.094 comentários,"Av. Antônio Sales, 2960 - Dionísio Torres, For...",$$,NaN,cafeteria,1.094,0.0,0.0
2,Bardo Café,4.9,22 comentários,"R. Amélia Benebien, 471 - Papicu, Fortaleza - ...",Cafeteria,NaN,cafeteria,22,0.0,0.0
3,"Go Coffee: Cafés, Chás, Frappes, Chocolate Que...",4.6,62 comentários,"R. Pereira Valente, 578 - Meireles, Fortaleza ...",Cafeteria,10 min,cafeteria,62,10.0,10.0
4,Le Pain Le Café | Aldeota,4.4,1.277 comentários,"Rua Professor Dias da Rocha, 670 - Meireles, F...",$$$,"20 min a 1,5 h",cafeteria,1.277,20.0,20.0
...,...,...,...,...,...,...,...,...,...,...
1145,Ceará,0.0,NaN,"R. Lucas Maia, 170 - Canindezinho, Fortaleza -...",NaN,NaN,drive thru,0,0.0,0.0
1146,Fortaleza,0.0,NaN,"Jardim Violeta, Zone 1, Uruguatuba, Colonia",NaN,NaN,drive thru,0,0.0,0.0
1147,Hospital Infantil Albert Sabin,4.2,391 comentários,"R. Tertuliano Sales, 544 - Vila União, Fortale...","Hospital infantil em Fortaleza, Ceará",NaN,drive thru,391,0.0,0.0
1148,Ilumina Led,5.0,9 comentários,"R. Pedro Pereira, 251 - Centro, Fortaleza - CE...","Loja de iluminação em Fortaleza, Ceará",NaN,drive thru,9,0.0,0.0


In [ ]:

geolocator=Nominatim(user_agent='geoapi')

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
def obter_coordenadas(endereco):
    try:
        local = geocode(endereco)
        if local:
            return pd.Series([local.latitude, local.longitude])
        else:
            return pd.Series([None, None])
    except:
        return pd.Series([None, None])

bruto[['lat', 'log']] = bruto['LOCAL'].apply(obter_coordenadas)

In [7]:
bruto.to_csv('dataFrameBruto.csv', index=False)